![Local image](./images/fastcampus/week06-a/week06-a-01.png)


> - 지난시간까지는 Value function을 학습하는 방법과 Policy function을 학습하는 방법에 대해 알아봤다. 이번에는 value function과 policy function을 동시에 학습해서 좀더 성능을 높이는 방법에 대해 알아볼 것이다.
- Policy function만 잘 학습하면 되는데 왜 value function도 같이 학습하는가? 그 이유는 variance reduction에 있다.

> - 유사한 예로 Return을 추정할 때 MC로 Sampling하면 sample의 variance가 매우 큰데 이를 개선하기 위한 TD, TD(lambda)의 접근을 사용 했었다. 즉 모든 time-step을 sampling하는 것이 아니라 초반 몇 time-step만 샘플링하고 나머지는 이전 시점에 추정한 값으로 대체하는 것이다.


> - Policy network를 학습할 때 사용하는 Policy Gradient Theorem에 포함된 $Q_{\pi_\theta}(s', a)$를 조절하여 variance를 줄일 수 있다. (baseline, GAE 등)

![Local image](./images/fastcampus/week06-a/week06-a-02.png)


![Local image](./images/fastcampus/week06-a/week06-a-03.png)


> - DPG(Deterministic Policy Gradient Algorithms), 2014
 - https://deepmind.com/research/publications/deterministic-policy-gradient-algorithms/


> - DDPG(Deep Deterministic Policy Gradients), 2016 - Deepmind
 - Continuous control with deep reinforcement learning
 - https://arxiv.org/abs/1509.02971
 
 
> - GAE(Generalized Advantage Estimation), 2016 - OpenAI
 - High-Dimensional Continuous Control Using Generalized Advantage Estimation
 - https://arxiv.org/abs/1506.02438

<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

![Local image](./images/fastcampus/week06-a/week06-a-04.png)


> - Policy Optimization에서는 return을 최대화 하도록 PG 방향으로 policy parameter를 업데이트 했었다. 이때 PG의 $Q_{\pi_\theta}(s', a)$는 sample return($G_t = \sum \gamma^t R_t$)으로 대체 했었다.
 - 이 부분을 별도의 Q-network로 학습시켜 사용하는 방법이 DDPG이고
 - 어떤 state-independent한 함수 $b$(baseline)를 이용해 $G_t - b$를 사용하는 방법과
 - $R_t + \gamma V(S_{t+1} - V(S_t)$를 사용하는 GAE라는 방법이 있다.

![Local image](./images/fastcampus/week06-a/week06-a-05.png)


> - Return을 샘플링할 때 MC를 사용하면 bias를 적지만 variance가 크다.(매 time-step마다 randomness가 작용하므로)
- 반면 TD를 사용하면 bias는 커지지만 variance는 작다. (randomness는 TD-step만큼만 있으므로)

![Local image](./images/fastcampus/week06-a/week06-a-06.png)


> Variance를 줄이는 방법중 하나로 $G_t$에서 baseline($b$)를 빼주는 방법이 있다. 
- $b$를 빼면 policy gradient가 틀어진다거나 하지는 않을까? - policy gradient에서 $b$만 포함된 항을 계산해보면 0이 된다. 상수항의 expectation이 0이므로 gradient의 방향을 바꾸지 않는 것이다. 하지만 variance를 줄이는 효과는 있다.
- 그렇다면 variance를 가장 많이 줄일 수 있는 $b$값은 무엇일까?

![Local image](./images/fastcampus/week06-a/week06-a-07.png)


> policy gradient의 variance를 최소화하는 b를 찾기 위해 $Var[\text{policy gradient}]$를 $\mathbb{E}[\cdot^2] - \mathbb{E}[\cdot]^2$ 형태로 정리하고, $b$에 대해 미분한 값을 0으로 놓고 정리한다.


![Local image](./images/fastcampus/week06-a/week06-a-08.png)


![Local image](./images/fastcampus/week06-a/week06-a-09.png)


> policy gradient의 variance를 최소화하는 $b$를 찾을 수 있다.

![Local image](./images/fastcampus/week06-a/week06-a-10.png)


> - 22:25

> - $b$가 $s$에 대한 함수라 하더라도 gradient의 방향이 바뀌지 않는다.

> $$\begin{align}
\sum_{s'}\sum_a \nabla_\theta \pi_\theta(a \mid s') b(s) \rho_{\pi_\theta}(s') &= \sum_{s'}\rho_{\pi_\theta}(s') b(s) \sum_a \nabla_\theta \pi_\theta(a \mid s') \\
&= \sum_{s'}\rho_{\pi_\theta}(s') b(s) \nabla_\theta \sum_a \pi_\theta(a \mid s') \\
&= \sum_{s'} \rho_{\pi_\theta}(s') b(s) \nabla_\theta 1 \\
&= 0 \end{align}$$

<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

![Local image](./images/fastcampus/week06-a/week06-a-11.png)


> - GAE는 Advantage를 구할 때 TD($\lambda$)의 아이디어를 적용한 것이다. TD($\lambda$)에서는 1-step TD, 2-step TD, ...의 지수평균을 타겟으로 사용했다. 마찬가지로 GAE에서는 1-step advantage $A^{(1)}_t$, 2-step advantage $A^{(2)}_t$, ...의 지수평균 $A^{(\lambda)}_t$을 새로운 avdantage로 사용한다.


> - $A^{(\lambda)}_t$를 구하는 과정은 아래와 같다.

![Local image](./images/fastcampus/week06-a/week06-a-12.png)


> - $\delta_t$는 1-step advantage이자 1-step TD error
> - 두번째 줄은 $\delta_{t+1}$

![Local image](./images/fastcampus/week06-a/week06-a-13.png)


![Local image](./images/fastcampus/week06-a/week06-a-14.png)


> - $A^{(\lambda)}_t = \delta_t + \gamma \lambda \delta_{t+1} + \gamma^2 \lambda^2 \delta_{t+2} + \cdots = \sum_{k=0} (\gamma \lambda)^k \big( R_{t+k+1} + \gamma V(s_{t+k+1}) - V(s_{t+k}) \big)$
- 결국 trajectory를 뽑아서 구한 1, 2, 3, ... step TD error($\delta_t, \delta_{t+1}, \cdots$)를 이용해 $A^{(\lambda)}_t$를 구할 수 있다.
- 현재 step의 advantage와 다음 step의 advantage의 관계로 표현할수도 있다. 이건 나중에 마지막 time-step부터 거꾸로 advantage를 계산할때 사용된다.
$$A^{(\lambda)}_t = \delta_t + \gamma \lambda A^{(\lambda)}_{t+1}$$


> - 상수 $\lambda$를 조절해서 각 time-step TD error가 advantage에 주는 영향을 조절할 수 있다. $\lambda$를 작게 조절하면 뒤 time-step TD error의 영향이 작아지고 variance가 작아지고 bias가 커진다.

![Local image](./images/fastcampus/week06-a/week06-a-15.png)


> - 일반적으로 reward는 sparse하다. 즉 거의 마지막 time-step에 도달해야만 reward를 받고, 그 이전 time-step에는 reward가 모두 0이다. 그런데 마지막 time-step 이전의 행동들도 마지막 결과를 얻는데 기여를 했으므로 reward를 각 time-step에 분배할 필요가 있고, 이런 형태의 advantage $A^{(\lambda)}_t$는 그런 역할을 한다.
- 위 그림에서 마지막 time-step에서 받은 reward $R_{T+1}$을 점점 줄여서 인접한 time-step에 분배한다.

<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

# <font color='green'>On Policy Actor Critic</font>

![Local image](./images/fastcampus/week06-a/week06-a-16.png)


> - action space가 discrete하면 좌측과 같이 각 action에 대한 probability를 출력하는 network를 학습시킨다.
- 반면 action space가 continuous하면 우측과 같이 action probability를 gaussian이라 가정하고 이 분포의 $\mu$와 $\sigma$를 출력하는 network를 학습시킨다.(action probability 분포는 laplacian, mixture of gaussian 등 분포를 사용할수도 있다.)

> - 지난주까지는 Policy Gradient $\nabla_\theta \eta(\pi_\theta) = \sum_{s'}\sum_a \nabla_\theta \pi_\theta(a \mid s') Q_{\pi_\theta}(s', a) \rho_{\pi_\theta}(s')$의 $Q_{\pi_\theta}(s', a)$를 sample return $G_t$로 대체 했었다. 그런데 Original Actor-Critic에서는 Q-network를 학습시키는 방법을 사용했고, GAE가 소개된 이후에는 $Q_{\pi_\theta}(s', a)$를 GAE advantage로 대체한다.

> - [1]. 47:00

![Local image](./images/fastcampus/week06-a/week06-a-17.png)


> - Actor-Critic with GAE
 - training policy network and value network
 - using $G_t$ to train value network
 - using GAE($A_t^{(\lambda)}$) as advantage 

> - Policy Network: $\pi_{\theta}(\cdot \mid S_t)$
 - with optimize rule of $\min_\theta \frac{1}{N} \sum_i \log \pi_\theta(a_i \mid s_i) A_i^{(\lambda)}$
 
> - Value Network: $V_{\phi}(S_t)$
 - with optimize rule of $\min_\phi \frac{1}{N} \sum_i 1/2 \big(G_i - V_{\phi}(s_i)\big)^2$
 
 

> - Policy network와 Value network를 학습시키는 과정은 아래와 같다.
 1. 에피소드의 시작 상태 $s_0$를 결정
 2. Policy Network $\pi_{\theta}(\cdot \mid S_t)$에서 $a_0$을 샘플링
 3. 환경에 $s_0, a_0$을 주고 $s_1, r_1$을 관찰
 4. 에피소드가 끝날때까지 2, 3 과정을 반복
 5. $(s_0, G_0), (s_1, G_1), \cdots, (s_T, G_T)$를 계산하고 이를 이용해 Value Network $V_{\phi}(S_t)$의 $\phi$를 업데이트
 7. $\delta_0, \delta_1, \cdots, \delta_T$를 이용해 $A^{(\lambda)_i}$를 계산하고 이를 이용해 $\pi_{\theta}(\cdot \mid S_t)$의 $\theta$를 업데이트
 6. 2번으로 돌아가 과정을 반복

![Local image](./images/fastcampus/week06-a/week06-a-18.png)


> - Actor-Critic with TRPO & GAE
 - training policy network and value network.(Actor-Critic)
 - using $G_t$ to train value network.
 - using GAE($A_t^{(\lambda)}$) as advantage. (GAE)
 - using importance ratio and constraint.(TRPO)

![Local image](./images/fastcampus/week06-a/week06-a-19.png)


> - Actor-Critic with PPO & GAE
 - training policy network and value network.(Actor-Critic)
 - using $G_t$ to train value network.
 - using GAE($A_t^{(\lambda)}$) as advantage. (GAE)
 - using cripped importance ratio and constraint.(PPO)

![Local image](./images/fastcampus/week06-a/week06-a-20.png)


> - Policy Gradient $\nabla_\theta \eta(\pi_\theta) = \sum_{s'}\sum_a \nabla_\theta \pi_\theta(a \mid s') Q_{\pi_\theta}(s', a) \rho_{\pi_\theta}(s')$의 $Q_{\pi_\theta}(s', a)$를 GAE로 바꿨고, GAE 계산에 필요한 TD-error를 구하기 위해 Value network를 학습시켰다. 


> - 일반적으로 Policy Network가 빨리 학습된다. 그런데 Policy Network의 예측값이 유의미하려면 GAE에 사용되는 Value 값이 정확해야 한다. 이 때문에 Value Function이 먼저 학습되도록 Value Network의 step-size를 Policy Network의 그것보다 크게 디자인한다.


> - 그런데 이 방법들은 샘플을 뽑는 network(policy)와 학습시키는 network(policy)가 같기 때문에 **On Policy**이다. 이 때문에 지금 학습에 사용한 trajectory를 다시 사용할 수 없기 때문에 sample inefficient한 문제가 있다.

> (오타) min -> max

<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

# <font color='green'>Off Policy Actor Critic</font>

![Local image](./images/fastcampus/week06-b/week06-b-01.png)


> - On Policy는 sample inefficient한 문제가 있다. 이를 Off Policy로 바꾸면 학습에 사용됐던 샘플을 다시 이용하는 테크닉(replay buffer, PER, ...)을 사요알 수 있다.

> - time2~

![Local image](./images/fastcampus/week06-b/week06-b-02.png)


> - Off Policy로 학습하는 가장 기초적인 방법은 샘플링 정책 $b(a\mid s)$을 별도로 가져가고, 학습 정책 $\pi_\theta(a \mid s')$과 샘플링 정책의 차이를 보정하기 위해 Importance ratio $\frac{\pi_\theta(a \mid s')}{b(a \mid s')}$를 Policy Gradient에 곱해준다.

![Local image](./images/fastcampus/week06-b/week06-b-03.png)


> - $\rho_{\pi_\theta}$는 $\pi$를 따라 움직였을 때 state distribution이다. 그런데 $\rho_{\pi_\theta}$에서 state을 뽑는 것이 가능하지 않다. 하지만 $\rho_b$에서는 state을 뽑는것이 가능하다. --- 왜??
 - $b$를 따라 움직이고 있으므로 샘플 trajectory를 이용해 $\rho_b$를 구할 수 있다.

![Local image](./images/fastcampus/week06-b/week06-b-04.png)


> - $s' \sim \rho_{\pi_\theta}$를 $s' \sim \rho_{b}$로 바꿔준다. (state을 뽑는 분포를 바꾼다.) 이렇게 바꾼 gradient를 써도 policy가 개선되는 효과가 있다. 즉 state을 뽑는 분포와 무관하게 $\nabla_\theta \log \pi_\theta(a\mid s') Q_{\pi_\theta}(s', a)$가 Policy개선 효과를 갖는다.
 - 그런데 이런 트릭은 우리가 Q network를 학습하고 있기 때문에 가능한 것이다.

<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

# <font color='green'>On Policy DPG</font>

> - DPG references
 - Talk of Silver about DPG: http://techtalks.tv/talks/deterministic-policy-gradient-algorithms/61098/ (http://techtalks.tv/talks/deterministic-policy-gradient-algorithms/61098/icml/2014/mp4:24_pm1_trackB_David_Silver.mp4)
 - stackoverflow: https://stackoverflow.com/questions/42763293/what-is-the-advantage-of-deterministic-policy-gradient-over-stochastic-policy-gr

![Local image](./images/fastcampus/week06-b/week06-b-05.png)


> - Stochastic policy: $\pi_\theta(s, a) = p(a \mid s; \theta)$
- Deterministic policy: $a = \mu_\theta(s)$

> Discrete action space
- objective function
$$\eta(\pi_\theta) = \mathbb{E}[\sum_{t=0}^\infty \gamma^t R_t \mid \pi_\theta]$$
 - Policy Gradient
 $$\nabla_\theta \eta(\pi_\theta) = \frac{1}{1-\gamma} \sum_s \rho_{\pi_\theta}(s) \sum_a \nabla_\theta \pi_\theta(a \mid s) Q^{\pi_\theta}(s, a)$$

> Continuous action space
- objective function
$$J(\theta) = \mathbb{E}_s \bigg[ \int_a \pi_\theta(s, a) R(s, a) ~ da\bigg]$$
 - Policy Gradient
 $$\begin{align}
 \nabla_\theta J(\theta) &= \mathbb{E}_s \bigg[ \int_a \nabla_\theta \pi_\theta(s, a) Q^\pi (s, a) da \bigg] \\
 &= \mathbb{E}_{s, a} \bigg[ \nabla_\theta \log \pi_\theta (s, a) Q^\pi (s, a) \bigg]
 \end{align}$$

> - Continuous action space에서 gradient를 구하기 위해 위 식과 같이 'gradiet of policy' $\nabla_\theta \pi_\theta(s, a)$와 'Q-function' $Q^\pi(s, a)$를 곱한 것을 action space에 대해 적분해야 한다. 다행히도 직접 적분을 계산하지 않고 sampling으로 아래와 같이 update rule을 이용할 수 있다.
$$\Delta\theta = \alpha \nabla_\theta \log \pi_\theta(s, a) Q^W(s, a)$$

> - 2; 9:30

![Local image](./images/fastcampus/week06-b/week06-b-06.png)


> (오타: 다음과 같이 수정) Variance of gradient estimate $\rightarrow \infty$ as policy $\rightarrow$ deterministic

> - $\Delta\theta = \alpha \nabla_\theta \log \pi_\theta(s, a) Q^W(s, a)$와 같이 sampling을 이용해 $\theta$를 업데이트 한다. 그런데 stochastic policy의 variance가 작아지면 $\nabla_\theta \log \pi_\theta(s, a)$은 위 그림과 같이 $\mu_\theta(s)$ 근방에서 크게 튀었다가 크게 떨어지는 형태가 된다. 이 런값이 sampling으로 추정하기 때문에 추정치의 variance가 매우 커진다. 이런 경향을 policy가 deterministic에 가까워질수록 심해진다.
 - Stochastic policy를 사용하는 경우에도 결국은 policy의 variance가 작아지는 것을 원하기 때문에 위 상황은 문제가 될 수 있다.
 
 
> - 그렇다면 정말 Stochastic policy의 variance가 0에 가까워지는 경우의 Policy gradient의 closed form을 구할수는 없을까?

<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

![Local image](./images/fastcampus/week06-b/week06-b-07.png)


> Stochastic policy의 variance가 0에 가까워질때 $\nabla_\theta \log \pi_\theta(s, a) Q^\pi(s, a)$은 위 그림과 같다. 이걸 action space에 대해 적분하면 아래와 같이 정리된다. (단지 Q-function에 대한 미분이 된다.) ($\frac{f(x+h) - f(x-h)}{h}$ 은 action 축으로의 미분임.)
$$\frac{1}{\epsilon} \bigg( Q^{\mu}(s, \mu(s) + \epsilon)) - Q^\mu(s, \mu(s) - \epsilon) \bigg)$$

> 이 결과를 정리하면 아래의 Deterministic policy gradient가 된다. Stochastic PG와 비교해보면 $a$에 대한 적분이 사라졌다는 것을 알 수 있다.
$$\nabla_\theta J(\theta) = \mathbb{E}_s [\nabla_{\theta} \mu_\theta(s) \nabla_a Q^\mu(s, a) \mid_{a=\mu_\theta(s)}]$$

![Local image](./images/fastcampus/week06-b/week06-b-08.png)


> (오타) $Q^u$가 아니라 $Q^\mu$이다.    
mu_theta(s, a) 오타... mu_theta(s)가 되어야 함.

> - Policy gradient in the DPG
- update rule for policy parameter $\theta$

# <font color='green'>Off Policy DPG</font>

![Local image](./images/fastcampus/week06-b/week06-b-09.png)


> - Deterministic Policy를 사용할때 한가지 문제는 exploration이 제한된다는 것이다. exploration을 위해 off-policy 기법을 이용한다.

![Local image](./images/fastcampus/week06-b/week06-b-10.png)


> objective function in the off-policy DPG

<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

![Local image](./images/fastcampus/week06-b/week06-b-11.png)


> - Critic is just a Q-learning network.

![Local image](./images/fastcampus/week06-b/week06-b-12.png)


> - Actor is updating policy network with Off-Policy DPG, $Q$ value is extracted from critic.


> - Off-Policy learning임에도 불구하고 Importance Sampling이 필요하지 않다.
- Deterministic PG는 Stochastic PG의 특별한 케이스라는 것이 증명되어 있다. 따라서 Stochastic PG에 적용되는 많은 기법들을 Deterministic PG에도 그대로 사용할 수 있다.

![Local image](./images/fastcampus/week06-b/week06-b-13.png)


> - x 축은 time-steps
- y 축은 cost
- 녹색은 Stochastic Actor-Critic(SAC-B)
- 빨간색은 Deterministic Actor-Critic(COPDAC-B)

![Local image](./images/fastcampus/week06-b/week06-b-14.png)


![Local image](./images/fastcampus/week06-b/week06-b-15.png)


<hr style="border: dashed rgb(0,255,0) 1.0px;background-color: rgb(170,0,0);height: 5.0px;"/>

# <font color='green'>DDPG</font>

![Local image](./images/fastcampus/week06-b/week06-b-16.png)


![Local image](./images/fastcampus/week06-b/week06-b-17.png)


![Local image](./images/fastcampus/week06-b/week06-b-18.png)


![Local image](./images/fastcampus/week06-b/week06-b-19.png)


> RL에서는 batch normalization이 별로 효과 없더라..

![Local image](./images/fastcampus/week06-b/week06-b-20.png)


![Local image](./images/fastcampus/week06-b/week06-b-21.png)


![Local image](./images/fastcampus/week06-b/week06-b-22.png)


![Local image](./images/fastcampus/week06-b/week06-b-23.png)


![Local image](./images/fastcampus/week06-b/week06-b-24.png)


![Local image](./images/fastcampus/week06-b/week06-b-25.png)


![Local image](./images/fastcampus/week06-b/week06-b-26.png)


![Local image](./images/fastcampus/week06-b/week06-b-27.png)
